# **`Knowledge Generation`**

- 크롤링(최신 뉴스기사 5개)추출 → 연관성 높은 기사 필터링 → 생성형 LLM으로 넘김
- 간단한 파이썬 코드와 GPT모델

## 크롤링 추출 단계

newspaper3k 사용

In [1]:
!pip install newspaper3k

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 11.1 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 10.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 11.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 11.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.6/97.6 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 785.1/785.1 kB 10.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 9.9 MB/s eta 0:00:00

In [3]:
!pip install lxml_html_clean

In [6]:
import requests
from bs4 import BeautifulSoup
from newspaper import Article
import pandas as pd

In [7]:
# 네이버 뉴스 검색 결과에서 뉴스 기사의 URL을 가져오는 함수
def get_news_urls(query):
    # 네이버 뉴스 검색 URL 구성
    search_url = f"https://search.naver.com/search.naver?where=news&ie=utf8&sm=nws_hty&query={query}"
    # 사용자 에이전트와 문자 인코딩 설정을 포함한 HTTP GET 요청을 보냄
    response = requests.get(search_url, headers={'User-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.114 Safari/537.36','Accept-Charset': 'utf-8'})
    if response.status_code == 200:
        # 응답으로 받은 HTML에서 BeautifulSoup을 이용해 뉴스 기사의 URL을 추출
        soup = BeautifulSoup(response.text, 'html.parser')
        news_elements = soup.find_all('a', class_='news_tit')
        news_urls = [element['href'] for element in news_elements]
        return news_urls
    else:
        print("Failed to fetch news URLs")
        return []

In [8]:
# 주어진 URL 목록에서 각 뉴스 기사를 스크랩하고 정보를 추출하는 함수
def scrape_news_articles(news_urls):
    articles_data = []
    for url in news_urls:
        try:
             # newspaper3k 라이브러리를 사용하여 기사를 다운로드하고 파싱
            article = Article(url)
            article.download()
            article.parse()
            
            # 추출된 정보를 딕셔너리 형태로 저장
            article_data = {
                'url': url,
                'title': article.title,
                'authors': article.authors,
                'publish_date': article.publish_date,
                'text': article.text
            }
            articles_data.append(article_data)
        except Exception as e:
            print(f"Failed to scrape article from {url}: {e}")

    return articles_data

In [9]:
# 추출된 기사 정보를 CSV 파일로 저장하는 함수
def save_to_csv(articles_data, filename):
    df = pd.DataFrame(articles_data)
    df.to_csv(filename, index=False)


### 지하철 파업 크롤링 (subway strike)

In [14]:
if __name__ == "__main__":
    # 첫 번째 쿼리: "지하철 파업"
    query = "지하철 파업"
    news_urls = get_news_urls(query)
    articles_data = scrape_news_articles(news_urls)
    save_to_csv(articles_data, "news_articles_subway_strike.csv")

    for article_data in articles_data:
        print("URL:", article_data['url'])
        print("Title:", article_data['title'])
        print("Authors:", article_data['authors'])
        print("Publish Date:", article_data['publish_date'])
        print("Text:", article_data['text'][:])
        print("\n")

    # 두 번째 쿼리: "지하철 연착"
    query = "지하철 연착"
    news_urls = get_news_urls(query)
    articles_data = scrape_news_articles(news_urls)
    save_to_csv(articles_data, "news_articles_subway_delay.csv")

    for article_data in articles_data:
        print("URL:", article_data['url'])
        print("Title:", article_data['title'])
        print("Authors:", article_data['authors'])
        print("Publish Date:", article_data['publish_date'])
        print("Text:", article_data['text'][:])  # Print only first 200 characters of the article text
        print("\n")


Failed to scrape article from https://www.yna.co.kr/view/AKR20240404075400004?input=1195m: Article `download()` failed with HTTPSConnectionPool(host='www.yna.co.kr', port=443): Max retries exceeded with url: /view/AKR20240404075400004?input=1195m (Caused by SSLError(SSLError(1, '[SSL: UNSAFE_LEGACY_RENEGOTIATION_DISABLED] unsafe legacy renegotiation disabled (_ssl.c:1006)'))) on URL https://www.yna.co.kr/view/AKR20240404075400004?input=1195m
Failed to scrape article from https://www.yna.co.kr/view/AKR20240328032501004?input=1195m: Article `download()` failed with HTTPSConnectionPool(host='www.yna.co.kr', port=443): Max retries exceeded with url: /view/AKR20240328032501004?input=1195m (Caused by SSLError(SSLError(1, '[SSL: UNSAFE_LEGACY_RENEGOTIATION_DISABLED] unsafe legacy renegotiation disabled (_ssl.c:1006)'))) on URL https://www.yna.co.kr/view/AKR20240328032501004?input=1195m
URL: https://www.khan.co.kr/local/Seoul/article/202404071500021
Title: [어떻게 생각하십니까?] 파업 때 지하철 ‘운행률’ 높이고, 버스는

In [15]:
df=pd.read_csv("news_articles_subway_strike.csv")
df.head()

url  \
0  https://www.khan.co.kr/local/Seoul/article/202...   
1  https://www.donga.com/news/Society/article/all...   
2  https://www.donga.com/news/Society/article/all...   
3  https://www.newsis.com/view/?id=NISX20240328_0...   
4  https://news.kbs.co.kr/news/pc/view/view.do?nc...   

                                               title authors  \
0  [어떻게 생각하십니까?] 파업 때 지하철 ‘운행률’ 높이고, 버스는 ‘필수공익사업’으로?      []   
1               [단독]“서울 지하철 파업해도 퇴근시간 운행률 80% 유지할 것”      []   
2          [단독]서울 지하철 파업해도 출퇴근시간 운행률 80% 이상 유지 방안 추진      []   
3                    서울 버스파업…"출근시간 두 배" "지하철 지옥"[현장]      []   
4                                             KBS 뉴스      []   

                publish_date  \
0  2024-04-07 15:00:00+09:00   
1        2024-04-04 12:00:00   
2        2024-04-04 12:00:00   
3  2024-03-28 12:51:29+09:00   
4                        NaN   

                                                text  
0  지난달 12년 만에 서울 시내버스가 멈췄다. 노사 교섭 실패로 노조가 파업에 들어가...  
1  필수유지업무 노사협정 변경 추진\n\n5~8호선 운행률 맞춰 퇴근대란 예방\n\n노...  
2  서울 지하철 파업 시 출퇴근 시간대에도 열차 운행률을 80% 이상 유지하도록 하는 ...  
3  [서울=뉴시스] 김선웅 기자 = 서울 시내버스가 노사 협상 결렬로 오전 4시를 기해...  
4  11시간가량 진행된 서울 시내버스 파업이 노사 협상 타결로 종료됐습니다. 버스는 정...

In [16]:
df=pd.read_csv("news_articles_subway_delay.csv")
df.head()

url  \
0  https://www.newsis.com/view/?id=NISX20240319_0...   
1  http://sports.chosun.com/news/ntype.htm?id=202...   
2  https://www.dailian.co.kr/news/view/1337742/?s...   
3  https://www.hankookilbo.com/News/Read/A2024022...   
4  http://www.yonhapnewstv.co.kr/MYH2024022201990...   

                                           title authors  \
0               또 멈춘 수도권지하철…"운행차량 칸 절반이상 20년 넘어"      []   
1                                            NaN      []   
2  서울지하철 정시도착률 살펴보니…5·6·7호선 '100점' [데일리안이 간다 36]      []   
3         "대체 언제 따뜻해지냐"… 폭설에 지하철 연착되고, 패딩 다시 꺼냈다      []   
4                                         연합뉴스TV      []   

                publish_date  \
0  2024-03-20 11:29:41+09:00   
1                        NaN   
2  2024-03-12 05:07:00+09:00   
3  2024-02-22 13:04:08+09:00   
4  2024-02-22 09:28:41+00:00   

                                                text  
0  [과천=뉴시스] 김금보 기자 = 최근 서울 지하철 1호선·3호선 등에서의 열차 고장...  
1                                                NaN  
2  1·3·4호선 출근시간 지나도 제 시간에 오는 열차 거의 없어…열차시간표 무용지물 ...  
3  출근길 지하철 2·5·7호선 지연\n\n눈길 교통사고 등 미끄러짐 사고\n\n"식목...  
4  브라우저가 video 태그를 지원하지 않습니다.\n\n죄송하지만 다른 브라우저를 사...

In [19]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import re

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

def clean_text(text):
    # HTML 태그 제거
    clean_text = re.sub('<.*?>', '', text)
    # 이모지 제거
    clean_text = clean_text.encode('ascii', 'ignore').decode('ascii')
    return clean_text

def tokenize_text(text):
    # 문장을 단어로 토큰화
    tokens = word_tokenize(text)
    return tokens

def remove_stopwords(tokens):
    # NLTK의 불용어 리스트 불러오기
    stop_words = set(stopwords.words('english'))
    # 불용어 제거
    filtered_tokens = [word for word in tokens if word.lower() not in stop_words]
    return filtered_tokens

def stem_words(tokens):
    # 어간 추출
    stemmer = PorterStemmer()
    stemmed_tokens = [stemmer.stem(word) for word in tokens]
    return stemmed_tokens

def lemmatize_words(tokens):
    # 표제어 추출
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return lemmatized_tokens

def preprocess_text(text):
    # 결측값 처리
    if pd.isnull(text):
        return ""
    
    cleaned_text = clean_text(text)
    tokens = tokenize_text(cleaned_text)
    filtered_tokens = remove_stopwords(tokens)
    stemmed_tokens = stem_words(filtered_tokens)
    lemmatized_tokens = lemmatize_words(filtered_tokens)
    return lemmatized_tokens

def preprocess_csv(csv_filename):
    # CSV 파일을 DataFrame으로 읽어옵니다.
    df = pd.read_csv(csv_filename)
    
    # 텍스트 전처리 적용
    df['text'] = df['text'].apply(preprocess_text)
    
    return df

# CSV 파일에서 텍스트 전처리 적용
delay_df = preprocess_csv("news_articles_subway_delay.csv")
strike_df = preprocess_csv("news_articles_subway_strike.csv")

print("Preprocessed Data for Delay Articles:")
print(delay_df.head())

print("\nPreprocessed Data for Strike Articles:")
print(strike_df.head())


Preprocessed Data for Delay Articles:
                                                 url  \
0  https://www.newsis.com/view/?id=NISX20240319_0...   
1  http://sports.chosun.com/news/ntype.htm?id=202...   
2  https://www.dailian.co.kr/news/view/1337742/?s...   
3  https://www.hankookilbo.com/News/Read/A2024022...   
4  http://www.yonhapnewstv.co.kr/MYH2024022201990...   

                                           title authors  \
0               또 멈춘 수도권지하철…"운행차량 칸 절반이상 20년 넘어"      []   
1                                            NaN      []   
2  서울지하철 정시도착률 살펴보니…5·6·7호선 '100점' [데일리안이 간다 36]      []   
3         "대체 언제 따뜻해지냐"… 폭설에 지하철 연착되고, 패딩 다시 꺼냈다      []   
4                                         연합뉴스TV      []   

                publish_date  \
0  2024-03-20 11:29:41+09:00   
1                        NaN   
2  2024-03-12 05:07:00+09:00   
3  2024-02-22 13:04:08+09:00   
4  2024-02-22 09:28:41+00:00   

                                                text  
0  [[, =, ], =, 

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
